In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
%matplotlib inline

In [2]:
gzip = pd.read_parquet('BMTC.parquet.gzip')
input = pd.read_csv('Input.csv')
ground_truth = pd.read_csv('GroundTruth.csv')

In [3]:
gzip['Time'] = pd.to_datetime(gzip['Timestamp'])
gzip['Time'] = gzip['Time'].dt.time

In [4]:
a = (gzip.groupby(by = 'BusID', as_index=False)['Speed'].sum())
Dropping_bus_id = (a[a['Speed'] == 0])

In [6]:
#The buses which moved only once in a short time span in the interval of 12 hrs
#[150218251, 150218847, 150219604, 150219952]

In [5]:
gzip.drop(gzip[gzip['BusID'] == 150218251].index.values, inplace=True)
gzip.drop(gzip[gzip['BusID'] == 150218847].index.values, inplace=True)
gzip.drop(gzip[gzip['BusID'] == 150219604].index.values, inplace=True)
gzip.drop(gzip[gzip['BusID'] == 150219952].index.values, inplace=True)

In [7]:
gzip.drop(Dropping_bus_id.index.values, inplace=True)
gzip.set_index(np.arange(0, len(gzip)), inplace=True)

In [8]:
def distance(lat1, long1, lat2, long2):
    R = 6371e3
    phi1 = lat1 * (math.pi)/180
    phi2 = lat2 * (math.pi)/180
    phi = (lat2 - lat1) * (math.pi) / 180
    lamb = (long2 - long1) * (math.pi) / 180
    
    a = math.sin(phi/2) * math.sin(phi/2) + math.cos(phi1) * math.cos(phi2) * math.sin(lamb/2) * math.sin(lamb/2)
    c = 2* math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d/1000

In [10]:
df1 = gzip.loc[gzip["Speed"] != 0]


In [11]:
lat_1_ = []
lat_2_ = []
for bus in np.unique(df1['BusID']):
        lat_1 = np.random.choice(a = df1['Latitude'],size = (400, 1), replace = True)
        for j in range(len(lat_1)):
            lat_1_.append(lat_1[j])

        lat_2 = np.random.choice(a = df1['Latitude'],size = (400, 1), replace = True)
        for j in range(len(lat_2)):
            lat_2_.append(lat_2[j])

In [12]:
df1 = df1.set_index(np.arange(len(df1)))

In [13]:
df_final_n1 = pd.DataFrame()
df_final_n2 = pd.DataFrame()

#df_final['bus_id'] = bus_id
df_final_n1['lat1'] = lat_1_
#df_final['long1'] = long1
df_final_n2['lat2'] = lat_2_
#df_final['long2'] = long2
#df_final['time'] = time

df_final_n1['latitude 1'] = df_final_n1['lat1'].apply(lambda x : float(x))
df_final_n1.drop(labels=['lat1'], axis = 1, inplace=True)
df_final_n2['latitude 2'] = df_final_n2['lat2'].apply(lambda x : float(x))
df_final_n2.drop(labels=['lat2'], axis = 1, inplace=True)

In [14]:
lat_1 = pd.merge(left = df1, right = df_final_n1, left_on='Latitude', right_on= 'latitude 1')
lat_2 = pd.merge(left = df1, right = df_final_n2, left_on='Latitude', right_on= 'latitude 2')

In [15]:
lat_1.drop_duplicates(subset=['Latitude', 'Longitude'], inplace=True)
lat_2.drop_duplicates(subset=['Latitude', 'Longitude'], inplace=True)
lat_1.set_index(np.arange(len(lat_1)), inplace=True)
lat_2.set_index(np.arange(len(lat_2)), inplace= True)

In [16]:
final = pd.DataFrame()
final['Latitude 1'] = lat_1['Latitude']
final['Longitude 1'] = lat_1['Longitude']
final['Timestamp 1'] = lat_1['Timestamp']
final['Latitude 2'] = lat_2['Latitude']
final['Longitude 2'] = lat_2['Longitude']
final['Timestamp 2'] = lat_2['Timestamp']

In [17]:
final

,Latitude 1,Longitude 1,Timestamp 1,Latitude 2,Longitude 2,Timestamp 2
0,13.074060,77.445267,2019-08-01 07:28:45,13.074558,77.445549,2019-08-01 07:25:16
1,13.074060,77.633720,2019-08-01 18:54:43,13.074558,77.437286,2019-08-01 15:59:26
2,13.074060,77.592506,2019-08-01 15:56:52,13.074558,77.504028,2019-08-01 15:56:46
3,13.074060,77.503525,2019-08-01 07:08:44,13.074558,77.437119,2019-08-01 09:44:32
4,13.074060,77.784119,2019-08-01 10:33:29,13.074113,77.445282,2019-08-01 07:25:26
...,...,...,...,...,...,...
2406160,13.091881,77.558899,2019-08-01 07:41:57,NaN,NaN,NaT
2406161,13.093278,77.560432,2019-08-01 07:42:38,NaN,NaN,NaT
2406162,13.091508,77.558556,2019-08-01 08:55:51,NaN,NaN,NaT
2406163,13.084355,77.559860,2019-08-01 09:00:39,NaN,NaN,NaT


In [18]:
final.dropna(inplace=True)

In [19]:
final['Duration'] = abs(final['Timestamp 2'] - final['Timestamp 1'])
final['Duration in seconds'] = final['Duration'].dt.seconds

In [20]:
dist = []
for i in range(0, len(final)):
    distance_ = distance(final['Latitude 1'][i], final['Longitude 1'][i], final['Latitude 2'][i], final['Longitude 2'][i])
    dist.append(distance_)
final['Distance'] = dist    

In [21]:
final.drop(np.where(final['Distance'] < 0.3)[0], axis = 0, inplace= True)
final

,Latitude 1,Longitude 1,Timestamp 1,Latitude 2,Longitude 2,Timestamp 2,Duration,Duration in seconds,Distance
1,13.074060,77.633720,2019-08-01 18:54:43,13.074558,77.437286,2019-08-01 15:59:26,0 days 02:55:17,10517,21.276324
2,13.074060,77.592506,2019-08-01 15:56:52,13.074558,77.504028,2019-08-01 15:56:46,0 days 00:00:06,6,9.583431
3,13.074060,77.503525,2019-08-01 07:08:44,13.074558,77.437119,2019-08-01 09:44:32,0 days 02:35:48,9348,7.192811
4,13.074060,77.784119,2019-08-01 10:33:29,13.074113,77.445282,2019-08-01 07:25:26,0 days 03:08:03,11283,36.700305
5,13.074060,77.423233,2019-08-01 11:00:36,13.074113,77.718834,2019-08-01 16:39:51,0 days 05:39:15,20355,32.017304
...,...,...,...,...,...,...,...,...,...
2405889,13.134246,77.667412,2019-08-01 14:51:53,13.069019,77.557526,2019-08-01 07:56:02,0 days 06:55:51,24951,13.936696
2405890,13.133813,77.667068,2019-08-01 14:52:03,13.085169,77.557724,2019-08-01 07:39:08,0 days 07:12:55,25975,13.018489
2405891,13.133813,77.667000,2019-08-01 16:09:00,13.095582,77.594666,2019-08-01 08:38:54,0 days 07:30:06,27006,8.912564
2405892,13.133040,77.652374,2019-08-01 14:56:22,13.092261,77.559471,2019-08-01 08:55:20,0 days 06:01:02,21662,11.035598


In [22]:
final['expected time'] = (final['Distance']/20)*60
final['Duration in mins'] = final['Duration in seconds']/60

In [24]:
final.drop(['Timestamp 1', 'Timestamp 2', 'Duration', 'Duration in seconds'], axis = 1, inplace=True)

In [25]:
final['time when speed = 7 kmph'] = (final['Distance']/7)*60
final['time when speed = 14 kmph'] = (final['Distance']/14)*60

In [26]:
final.drop(['expected time'], axis = 1, inplace= True)

In [31]:
final.set_index(np.arange(len(final)), inplace=True)

In [32]:
indx =[]
for i in range(len(final)):
    if(final['Duration in mins'][i] > final['time when speed = 7 kmph'][i]):
        indx.append(i)
    elif (final['Duration in mins'][i] < final['time when speed = 14 kmph'][i]):
        indx.append(i)

In [33]:
test_1 = final.drop(indx, axis = 0)

In [35]:
from xgboost import XGBRegressor as xgb

In [36]:
test_1 = final.drop(indx, axis = 0)
xgbr = xgb(learning_rate = 0.03, max_depth = 5)
final.columns

Index(['Latitude 1', 'Longitude 1', 'Latitude 2', 'Longitude 2', 'Distance',
       'Duration in mins', 'time when speed = 7 kmph',
       'time when speed = 14 kmph'],
      dtype='object')

In [37]:


X = test_1.drop(['Distance',
       'Duration in mins', 'time when speed = 7 kmph',
       'time when speed = 14 kmph'], axis=1)
y = test_1['Duration in mins']



In [38]:
xgbr.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.03, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [41]:
X_test = input.drop(['Unnamed: 0'], axis = 1)

In [42]:
predictions = xgbr.predict(X_test)

In [43]:
predictions

array([45.778023, 55.19887 , 46.183704, ..., 35.676937, 53.693893,
       66.07606 ], dtype=float32)

In [45]:
from sklearn.metrics import mean_absolute_error as mae
pd.DataFrame(data = (ground_truth['TT'], predictions)).T

,TT,Unnamed 0
0,2.833333,45.778023
1,1.500000,55.198872
2,21.250000,46.183704
3,2.000000,52.575573
4,35.733333,48.242481
...,...,...
1200,27.233333,49.211334
1201,26.750000,54.583889
1202,53.600000,35.676937
1203,59.616667,53.693893


In [46]:
mae(ground_truth['TT'], predictions)

23.973419176157083